In [1]:
import pandas as pd

In [2]:
import psycopg2

pg_host = "localhost"
pg_database = "graph"
pg_user = "postgres"
pg_password = "postgres"

# Establish PostgreSQL connection
pg_conn = psycopg2.connect(
    host=pg_host,
    database=pg_database,
    user=pg_user,
    password=pg_password
)
pg_cursor = pg_conn.cursor()

In [59]:
pg_cursor.execute("""
 select source_id, option_id, ds_source.sentence, ds_option.sentence from result_option_graph
 inner join datasets ds_source on ds_source.id = result_option_graph.source_id 
 inner join datasets ds_option on ds_option.id = result_option_graph.option_id  
 where "method" = 'COSINE' order by result_option_graph.source_id, result_option_graph.similarity desc
""")
result_graph = pg_cursor.fetchall()
df = pd.DataFrame(result_graph, columns=['source_id', 'option_id', 'source', 'option'])
df['is_similar'] = 0

In [4]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain, LLMChain

load_dotenv()

True

In [5]:
template1 = """
You are expert software analyst
Source : {source}
Option: 
{option}

Based on Source, which is the best Option that have similar feature of software requirement?
Answer must be 1-5 (based on from option number) or 0 (ff there are no similarities at all), dont give any other answer than 0-5

"""

prompt1 = ChatPromptTemplate.from_template(template1)

model = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv('OPEN_API_KEY'), temperature=0)

chain1 = prompt1 | model

chain_one = LLMChain(llm=model, prompt=prompt1, output_key="response")

overal_chain = SequentialChain(
    chains=[chain_one],
    input_variables=["source", "option"],
    output_variables=["response"]
)

/var/folders/80/kl6rpl3d2xj20wv4hyb0dbgc0000gn/T/ipykernel_4344/4121650547.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain_one = LLMChain(llm=model, prompt=prompt1, output_key="response")


In [6]:
def prompt_mrr_is_similar(df):
    for i in range(0, len(df), 5):
        option = ""
        idx = 1
        for j in range(i, i+5):
            option += f"{idx}. {df['option'][j]}\n"
            idx += 1
        message = overal_chain({ "source": df['source'][i], "option": option })
        response = message['response']
        df.loc[i, 'is_similar'] = df.iloc[i+int(response)]['option_id'] if response != '0' else 0
    return df

In [63]:
df_result = prompt_mrr_is_similar(df)

/var/folders/80/kl6rpl3d2xj20wv4hyb0dbgc0000gn/T/ipykernel_9475/201411453.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'e283462c-3365-4144-afb1-861bf8278375' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'is_similar'] = df.iloc[i+int(response)]['option_id'] if response != '0' else 0


In [65]:
filtered_df = df_result[df_result['is_similar'] != 0]
filtered_df.head()

,source_id,option_id,source,option,is_similar
0,0143aef8-853a-437b-ab2c-57351aff104a,0a898e3f-78ca-4fd0-8e1b-a2d61e38d251,Real-time accounting records generated for a p...,Real-time accounting records generated for a p...,e283462c-3365-4144-afb1-861bf8278375
5,03b19456-8ec4-4f61-868d-cbcbfb954531,4c816ace-5d99-45be-9b2e-7a9c0620b460,The System holds all the details of the all th...,It holds the details of all the godowns which ...,86692c68-a2b0-477d-a9a1-0478fa5d044a
10,075215f0-090b-414a-ab2a-9205de47723b,86692c68-a2b0-477d-a9a1-0478fa5d044a,The system shall enable the Sales Assistant to...,The system shall enable the Sales Assistant to...,4d3adc83-0b25-4573-a7e0-68f6b609a529
15,0955d54c-20b8-47b8-816a-c9c8092c4a22,14d70972-7f2f-4a38-b259-ed8587406305,The system shall enable the Sales Assistant to...,The system shall allow the Sales Assistant to ...,a5d5f14c-9de6-4cb9-81ee-4ca651604928
20,09ef99b8-518e-4284-885c-1cab53e39331,4908c8a4-55ba-4b16-994f-89368a1435b7,The system must provide options to export the ...,The system must allow the user to either displ...,be3b8c04-9894-4add-8970-d9935103c897


In [66]:
filtered_df.to_csv('data/result_graph_prompt.csv', index=False)

In [3]:
import pandas as pd
filtered_df = pd.read_csv('data/result_graph_prompt.csv')
filtered_df.head()

,source_id,option_id,source,option,is_similar
0,0143aef8-853a-437b-ab2c-57351aff104a,0a898e3f-78ca-4fd0-8e1b-a2d61e38d251,Real-time accounting records generated for a p...,Real-time accounting records generated for a p...,e283462c-3365-4144-afb1-861bf8278375
1,03b19456-8ec4-4f61-868d-cbcbfb954531,4c816ace-5d99-45be-9b2e-7a9c0620b460,The System holds all the details of the all th...,It holds the details of all the godowns which ...,86692c68-a2b0-477d-a9a1-0478fa5d044a
2,075215f0-090b-414a-ab2a-9205de47723b,86692c68-a2b0-477d-a9a1-0478fa5d044a,The system shall enable the Sales Assistant to...,The system shall enable the Sales Assistant to...,4d3adc83-0b25-4573-a7e0-68f6b609a529
3,0955d54c-20b8-47b8-816a-c9c8092c4a22,14d70972-7f2f-4a38-b259-ed8587406305,The system shall enable the Sales Assistant to...,The system shall allow the Sales Assistant to ...,a5d5f14c-9de6-4cb9-81ee-4ca651604928
4,09ef99b8-518e-4284-885c-1cab53e39331,4908c8a4-55ba-4b16-994f-89368a1435b7,The system must provide options to export the ...,The system must allow the user to either displ...,be3b8c04-9894-4add-8970-d9935103c897


In [4]:
for index, row in filtered_df.iterrows():
    pg_cursor.execute("UPDATE result_option_graph_prompt SET is_similar = true WHERE source_id = %s and option_id = %s", (row['source_id'], row['is_similar']))
    pg_conn.commit()

In [3]:
pg_cursor.execute("""
 select source_id, option_id, ds_source.sentence, ds_option.sentence from result_option_llm
 inner join datasets ds_source on ds_source.id = result_option_llm.source_id 
 inner join datasets ds_option on ds_option.id = result_option_llm.option_id  
 where "method" = 'EUCLIDEAN' order by result_option_llm.source_id, result_option_llm.similarity desc
""")
result_graph = pg_cursor.fetchall()
df = pd.DataFrame(result_graph, columns=['source_id', 'option_id', 'source', 'option'])
df['is_similar'] = 0

In [7]:
df_result = prompt_mrr_is_similar(df)

/var/folders/80/kl6rpl3d2xj20wv4hyb0dbgc0000gn/T/ipykernel_4344/201411453.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  message = overal_chain({ "source": df['source'][i], "option": option })
/var/folders/80/kl6rpl3d2xj20wv4hyb0dbgc0000gn/T/ipykernel_4344/201411453.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'fb944c0d-4c35-4d79-8f71-e919968f3ed7' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'is_similar'] = df.iloc[i+int(response)]['option_id'] if response != '0' else 0


In [8]:
filtered_df = df_result[df_result['is_similar'] != 0]
filtered_df.head()

,source_id,option_id,source,option,is_similar
0,009a2bef-0070-4f06-8a4e-dc306b17b7c2,2435debf-b31d-4219-87cb-7f18e8889b22,The system shall generate a detailed report of...,The system's report shall present a clear and ...,fb944c0d-4c35-4d79-8f71-e919968f3ed7
5,01379659-6e85-4055-a25b-c57056cbe842,535a65fa-a781-4c74-94fd-fc6483bff43c,The system must verify that the product associ...,The system must verify that the coupon ID is v...,59254e66-eb0d-405a-bb2b-0f208eb66837
10,0143aef8-853a-437b-ab2c-57351aff104a,0a898e3f-78ca-4fd0-8e1b-a2d61e38d251,Real-time accounting records generated for a p...,Real-time accounting records generated for a p...,85b9338c-0e0b-4922-8afe-edb9454ba980
15,01be1e05-bd55-4d02-a24c-3ca84e5b8a2d,dd2aee3e-1f21-4ed5-b78b-f54c1534979d,"Whether you run a Health & Beauty, Fashion & C...",Zoho Commerce has plenty of customizable layou...,3efbfde2-eb91-4b2c-95bc-af8abb0dcd19
20,03b19456-8ec4-4f61-868d-cbcbfb954531,cd2e6c72-c60e-412d-a9c5-e4fa3c7e43cc,The System holds all the details of the all th...,The system must display a list of all employee...,474418fc-7c58-4487-882a-ed37b9ca7bc1


In [9]:
filtered_df.to_csv('data/result_llm_prompt.csv', index=False)

In [10]:
import pandas as pd
filtered_df = pd.read_csv('data/result_llm_prompt.csv')
filtered_df.head()

,source_id,option_id,source,option,is_similar
0,009a2bef-0070-4f06-8a4e-dc306b17b7c2,2435debf-b31d-4219-87cb-7f18e8889b22,The system shall generate a detailed report of...,The system's report shall present a clear and ...,fb944c0d-4c35-4d79-8f71-e919968f3ed7
1,01379659-6e85-4055-a25b-c57056cbe842,535a65fa-a781-4c74-94fd-fc6483bff43c,The system must verify that the product associ...,The system must verify that the coupon ID is v...,59254e66-eb0d-405a-bb2b-0f208eb66837
2,0143aef8-853a-437b-ab2c-57351aff104a,0a898e3f-78ca-4fd0-8e1b-a2d61e38d251,Real-time accounting records generated for a p...,Real-time accounting records generated for a p...,85b9338c-0e0b-4922-8afe-edb9454ba980
3,01be1e05-bd55-4d02-a24c-3ca84e5b8a2d,dd2aee3e-1f21-4ed5-b78b-f54c1534979d,"Whether you run a Health & Beauty, Fashion & C...",Zoho Commerce has plenty of customizable layou...,3efbfde2-eb91-4b2c-95bc-af8abb0dcd19
4,03b19456-8ec4-4f61-868d-cbcbfb954531,cd2e6c72-c60e-412d-a9c5-e4fa3c7e43cc,The System holds all the details of the all th...,The system must display a list of all employee...,474418fc-7c58-4487-882a-ed37b9ca7bc1


In [11]:
for index, row in filtered_df.iterrows():
    pg_cursor.execute("UPDATE result_option_llm_prompt SET is_similar = true WHERE source_id = %s and option_id = %s", (row['source_id'], row['is_similar']))
    pg_conn.commit()